In [ ]:
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
import requests
from playwright.async_api import async_playwright
import time
import numpy as np
pd.set_option('display.max_colwidth', None)

In [ ]:
# Take today's date and format it - must be used in the URLs.

date = date.today()

date = date.strftime("%d.%m.%Y")

print(date)

In [ ]:
# Read in the links and update with today's date
df = pd.read_csv("data/links.csv")

df = df.replace(to_replace=r'01.01.2023', value= date, regex=True)
df

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()

url = df.iloc[0]['link']
await page.goto(url)

time.sleep(1)
await page.locator("#close-button").click()

In [ ]:
# Create an empty dataframe for the scrape.
total_df = pd.DataFrame()
total_df

In [ ]:
for index, row in df.iterrows():
    url = row['link']

    country = row['country']
    area = row['area']
    
    await page.goto(url)
    time.sleep(5)
    await page.get_by_text("PT60M").click()

    time.sleep(5)

    tables = pd.read_html(await page.content(), flavor = 'html5lib')
    data = tables[0]
    data.insert(0, 'country', country)
    data.insert(1, 'area', area)
    total_df = pd.concat([total_df, data], axis=0)
    
total_df

In [ ]:
total_df['date'] = pd.to_datetime('today').strftime("%m/%d/%Y")
total_df.columns = ['country', 'area', 'mtu', 'price(currency/mwh)', 'price(eur/mwh)', 'date']
total_df

# Standardize the missing values
total_df.replace({'-': np.nan}, inplace=True)
total_df.replace({'n/e': np.nan}, inplace=True)

# And reorder the columns
all_data = total_df[['date', 'country', 'area', 'mtu', 'price(currency/mwh)', 'price(eur/mwh)']]
all_data.head(50)

In [ ]:
all_data.to_csv("data/europaenergy_today.csv", index=False)